In [1]:
import torch

from omegaconf import OmegaConf


# sys.path.append(str(Path(__file__).parents[1]))
from scripts.arquitecturas import GNN_global, FCNN_global, GNN_Local
from scripts.Data_loader import load_net, load_data

import julia
julia.install()

from julia.api import Julia
jl = Julia(compiled_modules=False)

import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name 'GNN_global' from 'scripts.arquitecturas' (/home/iboero/Documents/GNN4OPF/Entrenar/scripts/arquitecturas.py)

## Levantar la red

In [4]:
# Load q shunt model
dt_v = "best"
dt_q = "best-05-21_22:21:19"

cfg_q = OmegaConf.load("./runs/uru/q_switch_shunt_opt/GNN_local/" + dt_q + "/config.yaml")
weights_dir_q = "./runs/uru/q_switch_shunt_opt/GNN_local/" + dt_q + "/weights/best_model.pt"

cfg_v = OmegaConf.load("./runs/uru/vm_pu_opt/GNN_local/" + dt_v + "/config.yaml")
weights_dir_v = "./runs/uru/vm_pu_opt/GNN_local/" + dt_v + "/weights/best_model.pt"
# Set device
torch.manual_seed(cfg_q.training.seed)
device = cfg_q.training.device

# Set network
num_nodes, num_gens, edge_index, edge_weights, feature_mask, net = load_net(cfg_q.data.red,cfg_q.data.red_path,cfg_q.data.target,device)

# Set model
if cfg_q.model.model == 'GNN_global':
    model_vm_pu = GNN_global(cfg_v.model.layers,edge_index,edge_weights,len(cfg_v.model.layers)-1,cfg_v.model.K,num_nodes,num_gens, feature_mask, cfg_v.model.batch_norm).to(device)
    model_q_shunt = GNN_global(cfg_q.model.layers,edge_index,edge_weights,len(cfg_q.model.layers)-1,cfg_q.model.K,num_nodes,num_gens, feature_mask, cfg_q.model.batch_norm).to(device)

elif cfg_q.model.model == 'FCNN_global':
    # Add in and out dimension
    cfg_q.model.layers[0] *= num_nodes
    model_q_shunt = FCNN_global(cfg_q.model.layers,len(cfg_q.model.layers)-1,num_nodes, feature_mask, cfg_q.model.batch_norm).to(device)
    cfg_v.model.layers[0] *= num_nodes
    model_vm_pu = FCNN_global(cfg_v.model.layers,len(cfg_v.model.layers)-1,num_nodes, feature_mask, cfg_v.model.batch_norm).to(device)

elif cfg_q.model.model == 'GNN_local':
    model_q_shunt = GNN_Local(cfg_q.model.layers,edge_index,edge_weights,len(cfg_q.model.layers)-1,cfg_q.model.K,feature_mask,num_nodes,cfg_q.model.batch_norm).to(device)
    model_vm_pu = GNN_Local(cfg_v.model.layers,edge_index,edge_weights,len(cfg_v.model.layers)-1,cfg_v.model.K,feature_mask,num_nodes,cfg_v.model.batch_norm).to(device)

elif cfg_q.model.model == 'FCNN_local':
    K = [0 for i in range(len(cfg_q.model.layers)-1)]
    model_q_shunt = GNN_Local(cfg_q.model.layers,edge_index,edge_weights,len(cfg_q.model.layers)-1,K,feature_mask,num_nodes,cfg_q.model.batch_norm).to(device)
    K = [0 for i in range(len(cfg_v.model.layers)-1)]
    model_vm_pu = GNN_Local(cfg_v.model.layers,edge_index,edge_weights,len(cfg_v.model.layers)-1,K,feature_mask,num_nodes,cfg_v.model.batch_norm).to(device)

# Cargo los pesos al modelo guardados en weights_dir +weights.pt
model_q_shunt.load_state_dict(torch.load(weights_dir_q))
model_vm_pu.load_state_dict(torch.load(weights_dir_v))

# Load data
train_loader_vm_pu, val_loader_vm_pu, test_loader_vm_pu = load_data(cfg_v.data.data_path, cfg_v.training.batch_size, cfg_v.data.normalize_X, cfg_v.data.normalize_Y,cfg_v.data.target,device)
train_loader_q_shunt, val_loader_q_shunt, test_loader_q_shunt = load_data(cfg_q.data.data_path, cfg_q.training.batch_size, cfg_q.data.normalize_X, cfg_q.data.normalize_Y,cfg_q.data.target,device)

FileNotFoundError: [Errno 2] No such file or directory: '/home/iboero/Documents/GNN4OPF/runs/uru/q_switch_shunt_opt/GNN_local/best-05-21_22:21:19/config.yaml'

In [1]:

def combined_loader(loader1, loader2):
    for (x1, y1), (x2, y2) in zip(loader1, loader2):
        assert torch.equal(x1, x2), "Inputs x from both loaders must be the same"
        yield x1, y1, y2

combined_data_loader = combined_loader(train_loader_q_shunt, train_loader_vm_pu)


NameError: name 'train_loader_q_shunt' is not defined